**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras almacenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
#configuración del laboratorio
# Ejecuta esta celda!
%load_ext autoreload
%autoreload 2
in_colab = True
import os
if not in_colab:
    import sys ; sys.path.append('../commons/utils/'); sys.path.append('../commons/utils/data')
else: 
    os.system('wget https://raw.githubusercontent.com/mariabda2/ML_2022/master/Labs/commons/utils/general.py -O general.py')
    from general import configure_lab3
    configure_lab3()
from lab3 import *
GRADER = part_2()
sns.set_context('talk')

# Laboratorio 3 - Parte 2. Comparación de metodos basados en árboles

A continuación se leen los datos de un problema de clasificación, usando los datos de analisis quimico de diferentes vinos. Como siempre [explora un poco el contenido y caracteristicas de nuestro conjunto de datos](https://archive.ics.uci.edu/ml/datasets/wine).



In [ ]:
x, y =  load_wine(return_X_y = True)
print(x.shape, "\n", x[0:3])
print(y[0:3])

## Ejercicio 1 Experimentos con Arboles de decisión

Debe consultar todo lo relacionado con la creación, entrenamiento y uso en predicción de este modelo usando la librería scikit-learn. En este enlace, se puede leer la documentación http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html.

En el notebook, ya se encuentra cargada la libreria:

```python
from sklearn.tree import DecisionTreeClassifier
```

Las siguientes preguntas abiertas, buscan verificar que se está haciendo un contraste con la librería y la teoría, por lo tanto procura incluir conceptos asociados y **NO** solo enfocarse en las descripciones de la documentación.

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Cuáles de los parámetros en la librería pueden ayudar a controlar el crecimiento del árbol de decisión (DT-Decision Tree)?
#@markdown ¿Cual de las opciones está mejor justificada?
 
#@markdown A) `criterion` es el parámetro que tiene más influencia ya que es el que decide medida para ganancia de información
 
#@markdown B) `max_depth` es el que tiene mayor influencia, controlando el número total de nodos del DT el resto de parámetros pueden influenciar pero no de manera directa.
 
#@markdown C) `max_depth`, `min_samples_split y `min_samples_leaf son importantes ya influencia en la creación de splits, aunque hay otros. `max_depth` es el que tiene mayor influencia, controlando el número total de nodos del DT.
 
#@markdown D) `max_depth` es el que tiene mayor influencia, controlando el número total de nodos del DT, luego  `max_features` que nos indica el máximo de características usadas.
 
#@markdown Selecciona dentro las lista desplegable
respuesta_1 = '' #@param ["", "A", "B", "C", "D"]

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Cuál es la diferencia principal entre `min_impurity_decrease` y `min_samples_split`? ¿cual tiene influencia en el número de nodos?
respuesta_2 = "" #@param {type:"string"}

En la siguiente celda se define una simulación para entrenar y validar un modelo usando los datos previamente cargados. Complete el código para usar como modelo de predicción un arbol de decisión.

<b>Note</b> que existe una clase para modelos de clasificación y otra para modelos de regresión:
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

Vamos a tener en cuenta lo siguiente:
1. dentro del código, ya se encuentra sugerida la metodologia de validación
2. la función va aceptar un parametro `str`, para seleccionar tipo de normalización.
    1. Estandar
    2. Minimo y maximos
3. **hacer uso explicito del nombre del parametro que se va usar**, por ejemplo, si se requeire asignar el parametro `max_features`  debemos llamar la libreria de esta manera: `DecisionTreeClassifier(max_features = 'auto')`
4. Vamos a configurar el arbol con la medida de impureza Entropia

In [ ]:
#ejercicio de código
def experimentar_dt( X, Y, depths,normalize):
    """funcion que realiza experimentos de arboles de decision
    Args:
        X: matriz con las caractersiticas
        Y: matriz de numpy con etiquetas
        depths: list[int] lista con la profundidad de arboles a experimentar
    normalize str: None, significa nignuna normalización. 'estandar'. 'min-max' los otros dos tipos
    retorna: dataframe con:
        - profunidad de los arboles
        - eficiencia de entrenamiento
        - desviacion de estandar eficiencia de entrenamiento
        - eficiencia de prueba
        - desviacion estandar eficiencia de prueba
    """
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for depth in depths:
        ## para almacenar los errores intermedios
        EficienciaTrain = []
        EficienciaVal = []
        for train, test in skf.split(X, Y):
            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #Normalizamos los datos
            # si la bandera esta en True
            scaler = None
            if normalize == 'estandar':
                # usa la clase adecuada
                scaler = ...()
                scaler.fit(Xtrain)

            elif normalize == 'min-max':
                # usa la clase adecuada
                scaler = ...()
            
            else:
                if idx < 1:
                    # solo imprimir la advertencia una vez POR FOLD
                    print("No se ejecuta niguna normalizacion/estandarizacion")

            if scaler is not None:
                scaler.fit(Xtrain)
                Xtrain= scaler.transform(Xtrain)
                Xtest = scaler.transform(Xtest)
    

            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            modelo = ...(max_depth=depth, criterion="entropy")
            modelo...(...)
            #predecir muestras de entrenamiento
            Ytrain_pred = modelo...(...)
            #predecir muestras de pruebas
            Ytest_pred = modelo...(X=...)
            #Evaluamos las predicciones del modelo con los datos de test
            EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
            EficienciaVal.append(np.mean(Ytest_pred.ravel() == Ytest.ravel()))

        resultados.loc[idx,'profunidad del arbol'] = depth
        resultados.loc[idx,'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx,'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
        idx= idx +1
        
    return (resultados)

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio1", experimentar_dt)

In [ ]:
# Realiza los experimentos para observar los efectos

resultados_dt = []
depths = [10,20,30,50]
normalizaciones = ['ninguna', 'estandar', 'min-max']

for normalize in normalizaciones:
    # usa tu funcion
    resultados_parciales = ... (X = x , Y = y, depths = depths, normalize=normalize)
    resultados_parciales['normalizacion'] = normalize
    resultados_dt.append(resultados_parciales)

resultados_dt = pd.concat(resultados_dt, ignore_index=True)

sns.catplot(data = resultados_dt, 
            x = 'profunidad del arbol', 
            y = 'eficiencia de prueba', 
            hue = 'normalizacion' ,
            kind = 'point', 
            aspect = 2)

In [ ]:
#@title Pregunta Abierta
#@markdown ¿Tiene algún efecto la normalización o estandarización de las variables en el desempeño del modelo de árboles de decisión?
#@markdown ¿Cual de las opciones está mejor justificada? Puedes volver a ejecutar experimentos
 
#@markdown A) Se observan que las diferencias no son muy grandes pero no hay patrón claro, si se vuelve a ejecutar los experimentos los efectos pueden variar, la normalización debe ser tratada como un parámetro más
 
#@markdown B) Se observan que las diferencias son grandes pero hay un patrón claro, si se vuelve a ejecutar los experimentos los efectos no varían, la normalización no tiene influencia ya que los DT no son sensibles a los diferentes rangos entre características
 
#@markdown C) Se observan que las diferencias no son muy grandes, si se vuelve a ejecutar los experimentos los efectos pueden variar, eso se debe a que no estamos configurando un `random_state`, sin embargo la normalización en los DT no tiene tanta influencia porque las comparaciones se hacen entre valores de una misma característica
 
#@markdown D) Se observan que la diferencias son pequeñas y hay un patrón claro, si se vuelve a ejecutar los experimentos los efectos pueden variar, eso se debe a que no estamos configurando un `random_state`, pero sabemos que la normalización en los DT no tiene influencia por que las comparaciones se hacen entre valores de una misma característica
 
#@markdown Selecciona dentro las lista desplegable
respuesta_3 = '' #@param ["", "A", "B", "C", "D"]

Vamos a explorar un poco la visualización de los arboles, reemplaza los mejores valores de parametros encontrados.

In [ ]:
# se usa para visualizar el DT
from sklearn import tree
# reemplaza el valor
mejor_profundidad = ...
modelo = DecisionTreeClassifier(max_depth=mejor_profundidad, criterion="entropy", ccp_alpha =...) # reemplaza el valor
modelo.fit(x, y)
tree.plot_tree(modelo,filled = True,class_names = classes_wines(), feature_names= features_wines())
print("ccp_alpha = 0.0")

In [ ]:
# se usa para visualizar el DT
from sklearn import tree
modelo = DecisionTreeClassifier(max_depth=mejor_profundidad, criterion="entropy", ccp_alpha =...) # reemplaza el valor
modelo.fit(x, y)
tree.plot_tree(modelo,filled = True,class_names = classes_wines(), feature_names=features_wines())
print("ccp_alpha = 0.5")

Observa la influencia de el parámetro `ccp_alpha`.  Investiga un poco acerca del concepto de "Poda" / Prunning en los DT.

In [ ]:
#@title Pregunta Abierta
#@markdown  ¿que relación tiene el parámetro `ccp_alpha` y el concepto de "poda" de arboles? ¿que efectos practicos tiene el podado de los arboles?
respuesta_4 = "" #@param {type:"string"}

## Ejercicio 2 Experimentos con Random Forest

En la siguiente celda se define una simulación para entrenar y validar un modelo usando los datos previamente cargados. Complete el código para usar como modelo de predicción un Random Forest. Debe consultar todo lo relacionado con la creación, entrenamiento y uso en predicción de este modelo usando la librería scikit-learn. Consultar aquí: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html.

En el notebook, ya se encuentra cargada la libreria:

```python
from sklearn.ensemble import RandomForestClassifier

```

<b>Note</b> que al igual que en el caso anterior, existe una clase para modelos de clasificación y otra para modelos de regresión: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

**Recordar hacer uso explicito del nombre del parametro que se va usar**, por ejemplo, si se requiere asignar el parametro `criterion`  debemos llamar la libreria  de esta manera: `RandomForestClassifier(criterion = 'gini')`.

Para nuestros experimentos vamos a configurar el RF para que el mínimo de muestras para considerar un nodo como hoja sea de 3.

In [ ]:
#ejercicio de código
def experimentar_rf(X, Y, num_trees,numero_de_variables):
    """funcion que realiza experimentos de random forest
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    num_trees: list[int]: lista con el número de arboles usado para el RF
    numero_de_variables list[int]: lista con variables para la selección del mejor umbral en cada nodo 
    retorna: dataframe con:
        -  numero de arboles usados
        -  variables para la selección del mejor umbral
        - eficiencia de entrenamiento
        - desviacion de estandar eficiencia de entrenamiento
        - eficiencia de prueba
        - desviacion estandar eficiencia de prueba
    """
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for trees in num_trees:
        for num_variables in numero_de_variables:
            ## para almacenar los errores intermedios
            EficienciaTrain = []
            EficienciaVal = []
            for train, test in skf.split(X, Y):
                Xtrain = X[train,:]
                Ytrain = Y[train]
                Xtest = X[test,:]
                Ytest = Y[test]
                #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
                modelo = ...(...)
                modelo...(...)
                #predecir muestras de entrenamiento
                Ytrain_pred = modelo...(...)
                #predecir muestras de pruebas
                YtestPred = modelo...(...)
                #Evaluamos las predicciones del modelo con los datos de test
                EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
                EficienciaVal.append(np.mean(YtestPred.ravel() == Ytest.ravel()))

            resultados.loc[idx,'número de arboles'] = trees
            resultados.loc[idx,'variables para la selección del mejor umbral'] = num_variables
            resultados.loc[idx,'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
            resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
            resultados.loc[idx,'eficiencia de prueba'] = np.mean(EficienciaVal)
            resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
            idx= idx +1
        print(f"termina para {trees} arboles")
        
    return (resultados)

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio2", experimentar_rf)

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [ ]:
arboles = [5,20,50,100, 150]
variables_seleccion = [2,5,10, x.shape[1]]
# reemplaza por la funcion
resultados_rf = ... (X=x, Y=y, num_trees = arboles ,numero_de_variables = variables_seleccion)
resultados_rf

In [ ]:
#@title Pregunta Abierta
#@markdown ¿qué relación tiene `sklearn.ensemble.BaggingClassifier`cuando los son Random forest entrenados con el parametro `max_features=13`?
respuesta_5 = "" #@param {type:"string"}

Vamos a comparar los resultados del RF y con el DT.

In [ ]:
print("diferencia promedio entre entrenamiento y prueba del DT", 
      resultados_dt['eficiencia de entrenamiento'].mean() - resultados_dt['eficiencia de prueba'].mean())

print("diferencia promedio entre entrenamiento y prueba del RF", 
      resultados_rf['eficiencia de entrenamiento'].mean()-resultados_rf['eficiencia de prueba'].mean())

de acuerdo a estos resultados, que modelo tuvo un mayor sobre entrenamiento?, ten presente esa diferencia para responder la siguiente pregunta abierta.

In [ ]:
#@title Pregunta Abierta
#@markdown ¿esperaba la diferencia que se observa entre las eficiencias entre entrenamiento y pruebas para el Random forest y el arbol de decisón? justifique 
respuesta_6 = "" #@param {type:"string"}

## Ejercicio 3 Experimentos con Gradient Boosted Trees

En la siguiente celda se define una simulación para entrenar y validar un modelo usando los datos previamente cargados. Complete el código para usar como modelo de predicción un Gradient boosted Tree. Debe consultar todo lo relacionado con la creación, entrenamiento y uso en predicción de este modelo usando la librería scikit-learn. Consultar aquí: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

En el notebook, ya se encuentra cargada la libreria:

```python
from sklearn.ensemble import GradientBoostingClassifier

```

Debemos configurar el árbol con un mínimo de tres (3) muestras para considerar una división de un nodo.

<b>Note</b> que al igual que en el caso anterior, existe una clase para modelos de clasificación y otra para modelos de regresión: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

**Recordar hacer uso explicito del nombre del parametro que se va usar**, por ejemplo, si se requeire asignar el parametro ` loss`  debemos llamar la libreria  de esta manera: `GradientBoostingClassifier(loss = 'deviance')`

In [ ]:
#ejercicio de código
def experimentar_gbt(num_trees, X, Y):
    """funcion que realiza experimentos de arboles de decision
    num_trees: list[int] lista con el número de arboles usado para el RF
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    retorna: dataframe con:
        - numero de arboles usados
        - eficiencia de entrenamiento
        - desviacion de estandar eficiencia de entrenamiento
        - eficiencia de prueba
        - desviacion estandar eficiencia de prueba
    """
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for trees in num_trees:
        ## para almacenar los errores intermedios
        EficienciaTrain = []
        EficienciaVal = []
        for train, test in skf.split(X, Y):
            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            modelo = ...(...)
            modelo...(...)
            #predecir muestras de entrenamiento
            Ytrain_pred = ...(...)
            #predecir muestras de pruebas
            Ytest_pred = modelo...(...)
            #Evaluamos las predicciones del modelo con los datos de test
            EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
            EficienciaVal.append(np.mean(Ytest_pred.ravel() == Ytest.ravel()))

        resultados.loc[idx,'número de arboles'] = trees
        resultados.loc[idx,'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx,'eficiencia de prueba'] =np.mean(EficienciaVal)
        resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
        idx= idx +1
        
    return (resultados)

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio3", experimentar_gbt)

In [ ]:
# ejecuta para realizar los experimentos
resultados_gbt = experimentar_gbt(arboles, x, y)
resultados_gbt

Vamos a graficar la eficiencia para el RF y el GBT en función del número de arboles.

In [ ]:
# se crea un df para agrupar los resultados
# y graficar las diferencias entre el GBT y el RF
rf_res = resultados_rf.groupby("número de arboles")['eficiencia de prueba'].mean().reset_index()
rf_res['Tipo'] = 'RF'
gbt_res = resultados_gbt.groupby("número de arboles")['eficiencia de prueba'].mean().reset_index()
gbt_res['Tipo'] = 'GBT'
data_to_plot= pd.concat([rf_res, gbt_res], ignore_index=True)
sns.relplot(data=data_to_plot, x= 'número de arboles', y = 'eficiencia de prueba', hue = 'Tipo', kind='line', aspect=1.5,height=3)

## Ejercicio 4 Tiempo de entrenamiento del RF y GBT

En nuestro último experimento, vamos a evaluar la influencia de las parametros del RF y del GBT en el tiempo de entrenamiento. 

Para ello vamos a crear una función para medir el tiempo de entrenamiento usando la instrucción `time.process_time()`.

Vamos crear la función, para poder evaluar la influencia de:
1. número de arboles
2. cantidad de variables a analizar por nodo

En el entrenamiento del RF y del GBT. 

**Notar**  
1. No vamos a dividir el conjunto, ya que el objetivo es evaluar el tiempo de entrenamiento y no la eficiencias del modelo
2. No calculamos las prediciones
3. **Recordar hacer uso explicito del nombre del parametro que se va usar**, por ejemplo, si se requeire asignar el parametro `criterion`  debemos llamar la libreria  de esta manera: `RandomForestClassifier(criterion = 'gini')`

In [ ]:
def time_rf_gbt_training(X, Y, num_trees, numero_de_variables, metodo):
    """funcion que realiza experimentos, para determinar la influencia
    del numero de arboles y de caracteristicas en el tiempo de entrenamiento
    del RF
    X: conjunto de datos para realizar los experimentos
    Y: conjunto de etiquetas de clase
    num_trees: List[int] lista con el número de arboles a evaluar
    num_variables: List[int] lista con el número variables a evaluar
    retorna: dataframe con:
    - número de arboles
    - variables para la selección del mejor umbral
    - tiempo de entrenamiento
    """
    resultados = pd.DataFrame()
    idx = 0
    
    for trees in num_trees:
        for variables in numero_de_variables:
            ## ejecutar 5 veces lo mismo
            ## para llegar a un tiempo más adecuado
            tiempos = []
            for i in range(5):
            ## llamar la 
                start = time.process_time()
                if metodo == 'rf':
                
                    modelo ...
                else:
                    modelo = ... 
                modelo.fit(X=X, y=Y)
                ## obtener tiempo 
                end = time.process_time()
                # append de la resta de fin y end
                tiempos.append( ... )
            resultados.loc[idx,'número de arboles'] = trees
            resultados.loc[idx,'variables para la selección del mejor umbral'] = variables
            resultados.loc[idx,'tiempo de entrenamiento'] = np...(...)
            resultados.loc[idx,'metodo'] = metodo
            idx = idx +1
    return(resultados)

In [ ]:
## la funcion que prueba tu implementacion
GRADER.run_test("ejercicio4", time_rf_gbt_training)

Vamos a dejar fijo el número de variables en 20 y variar los árboles en: [5,10,15,25, 50], completa el código para ver la grafica

In [ ]:
resultados_rf_time1 = time_rf_gbt_training(x, y, [5,10,15,25,50], [20], metodo = 'rf')
resultados_gbt_time1 = time_rf_gbt_training(x, y, [5,10,15,25,50], [20], metodo = 'gbt')

resultados_time = pd.concat([resultados_rf_time1, resultados_gbt_time1], ignore_index=True)

sns.relplot(data = resultados_time, x = 'número de arboles', y = 'tiempo de entrenamiento', hue = 'metodo', kind = 'line')

Y por ultimo Vamos a dejar fijo el número de árboles en 20 y el número de varaibles [5,10,15,20,40], completa el código para ver la grafica

In [ ]:
resultados_rf_time1 = time_rf_gbt_training(x, y, [20], [5,10,15,20,40], metodo = 'rf')
resultados_gbt_time1 = time_rf_gbt_training(x, y, [20], [5,10,15,20,40], metodo = 'gbt')

resultados_time = pd.concat([resultados_rf_time1, resultados_gbt_time1], ignore_index=True)

sns.relplot(data = resultados_time, x = 'variables para la selección del mejor umbral', y = 'tiempo de entrenamiento', hue = 'metodo', kind = 'line')

In [ ]:
#@title Pregunta Abierta
#@markdown ¿qué parametro de los evaluados tiene una mayor influencia en los tiempos de entrenamiento? ¿hay diferencia entre el RF y GBT? justifique
respuesta_7 = "" #@param {type:"string"}

In [ ]:
GRADER.check_tests()

In [ ]:
#@title Integrantes
codigo_integrante_1 ='' #@param {type:"string"}
codigo_integrante_2 = ''  #@param {type:"string"}

----
esta linea de codigo va fallar, es de uso exclusivo de los profesores


In [ ]:
GRADER.grade()